# Compute the fermat potential prior

copy from Df_pot_prior, but more similar to prior_fermat_pot.ipynb
basically instead of creating it from the setting file, it is based on the mcmc results and then we create from that the uniform distribution for the prior of the lens parameters

In [2]:
# import of standard python libraries
import matplotlib.pyplot as plt
import numpy as np
import os
from lenstronomy.LensModel.lens_model import LensModel
import json
import corner
import sys
import argparse
import copy

#%matplotlib inline
from tools import *
from order_images import get_new_image_order
from mag_remastered import get_mag_mcmc
from fermat_pot_analysis import gen_mcmc_Df

In [1]:
# This parts give the mcmc of the fermat potential given the samples_mcmc prior


def get_mcmc_Df_prior(mcmc_prior,param_mcmc,setting,Df_boundaries=None,threshold_mcmc_points=100): 
    # mcmc_prior shape = param, n_points
    # param_mcmc = list of name of params
    # setting = setting module
    # Df_boundaries = [[min_AB,max_AB],[min_A..],..]
    
    mcmc_prior_Df = gen_mcmc_Df(mcmc_prior,param_mcmc,setting)  #D_AB, D_AC, D_AD, meaning Df_i - Df_A
    
    # cut points out of boundaries
    if Df_boundaries is not None:
        excluded_points=0
        mcmc_prior_Df_cut = []
        for mcmcdfi in mcmc_prior_Df:
            if not is_in_boundaries(mcmcdfi,Df_boundaries):
                excluded_points+=1
            else:
                mcmc_prior_Df_cut.append(mcmcdfi)
        if len(mcmc_prior_Df_cut)<threshold_mcmc_points:
            raise UserWarning("The excluded points are",excluded_points,", leaving only",len(mcmc_prior_Df_cut)," points in the final fermat MCMC. Run with larger n_points")
        
        # double check
        # for some reason necessary to eliminate 1 single point
        # that appeared where it was not supposed to
        for i in range(len(mcmc_prior_Df_cut)):
            if not is_in_boundaries(mcmc_prior_Df_cut[i],Df_boundaries):
                del mcmc_prior_Df_cut[i]
        return mcmc_prior_Df_cut
    
    return mcmc_prior_Df # shape: Df_ai, mcmc_steps
    #plot = corner.corner(mcmc_Df, labels=labels_Df, show_titles=True)
    #plot.savefig(savefig_path+"test_uniform_df.png")
    #print("test result in "+savefig_path+"test_uniform_df.png")

In [1]:
# This parts give the mcmc of the mag ratio the samples_mcmc prior
def get_mcmc_mag_prior(mcmc_prior,param_mcmc,setting,mag_boundaries=None,threshold_mcmc_points=100): 
    # mcmc_prior shape = param, n_points
    # param_mcmc = list of name of params
    # setting = setting module
    # mag_boundaries = [[min_AB,max_AB],[min_A..],..]
    
    CP= check_if_CP(setting)
    mcmc_mag = get_mag_mcmc(samples_mcmc,setting,CP)
  
    #I want to obtain the correct image order
    #########################################
    new_order = get_new_image_order(setting,starting_from_A=False)
    tmp_mcmc = np.transpose(mcmc_mag) 
    mcmc_prior_mag_rt = np.transpose([tmp_mcmc[i] for i in new_order]).tolist() #mu_AB, mu_AC, mu_AD, meaning mu_i / mu_A
    
    # cut points out of boundaries
    if mag_boundaries is not None:
        excluded_points=0
        mcmc_prior_mag_cut = []
        for mcmcMui in mcmc_prior_mag_rt:
            if not is_in_boundaries(mcmcMui,mag_boundaries):
                excluded_points+=1
            else:
                mcmc_prior_mag_cut.append(mcmcMui)
        if len(mcmc_prior_mag_cut)<threshold_mcmc_points:
            raise UserWarning("The excluded points are",excluded_points,", leaving only",len(mcmc_prior_mag_cut)," points in the final Mag MCMC. Run with larger n_points")
        
        # double check
        # for some reason necessary to eliminate 1 single point
        # that appeared where it was not supposed to
        for i in range(len(mcmc_prior_mag_cut)):
            if not is_in_boundaries(mcmc_prior_mag_cut[i],mag_boundaries):
                del mcmc_prior_mag_cut[i]
        return mcmc_prior_mag_cut
    
    return mcmc_prior_mag_rt # shape: mag_ai, mcmc_steps

In [ ]:
def is_in_boundaries(dfi,Df_boundaries):
    for IJ,Dfi_IJ in enumerate(dfi):
        if not Dfi_IJ>=Df_boundaries[IJ][0] or not Dfi_IJ<Df_boundaries[IJ][1]:            
            return False
    return True

In [ ]:
def get_prior(setting_module,npoints):
    params     = param_list(setting_module.__name__) #mcmc params like
    n_images   = count_images(params)
    setting    = setting_module.setting()
    kw_models  = get_kwargs_to_model(setting)
    mcmc_prior = []
    for ip,param in enumerate(params):
        if "image" in param:
            par_min,par_max = get_boundary_param(param,kw_models,ip,len(params),n_images)
        else:
            par_min,par_max = get_boundary_param(param,kw_models)
        mcmc_prior.append(np.random.uniform(par_min,par_max,npoints))
    mcmc_prior = np.transpose(mcmc_prior).tolist()
    return mcmc_prior, params
    

def param_list(setting_name):
    # TODO: find a way to do that even before running the mcmc
    # Easy way out, but depends on the modelling already been run:
    from get_res import get_mcmc_prm
    return get_mcmc_prm(setting_name)

def count_images(params):
    n_im_ra  = 0
    n_im_dec = 0
    for p in params:
        if "ra_image" in p:
            n_im_ra+=1
        elif "dec_image" in p:
            n_im_dec+=1
    if n_im_dec!=n_im_ra:
        raise RuntimeError("Number of coordinates for images in parameters not matching")
    return n_im_ra

def get_kwargs_to_model(setting):
    kw_to_model= {}
    #those are always present
    kw_to_model["lens_params"]       = setting.lens_params
    kw_to_model["ps_params"]         = setting.ps_params
    kw_to_model["lens_light_params"] = setting.lens_light_params  # at least the uniform
    try:
        kw_to_model["source_params"] = setting.source_params
    except:
        pass
    return kw_to_model

def get_boundary_param(param,kw_models,ip=None,nparam=None,n_images=4):
    kw_model,split_name = kw_model_from_param(kw_models,param)
    lower_kw = kw_model[3]
    upper_kw = kw_model[4]
    if split_name=="_image" and ip is None:
        raise RuntimeError("Give me the index of the parameter for the ps image boundaries")
    elif split_name=="_image" and nparam is None:
        raise RuntimeError("Give me the lenght of the parameters for the ps image boundaries")
    elif split_name=="_image":
        nimg = index_image(param,ip,nparam,n_images)
        par_min,par_max = lower_kw[0][param][nimg],upper_kw[0][param][nimg]
    else:
        index_model = int(param[-1])
        key_prm = param.split(split_name)[0] 
        par_min = lower_kw[index_model][key_prm]
        par_max = upper_kw[index_model][key_prm]
    return par_min,par_max

def kw_model_from_param(kw_models,param):
    for word in ["theta_E","e1","e2","gamma","gamma_ext","psi_ext"]:
        if word+"_lens" in param:
            return kw_models["lens_params"],"_lens"
    for word in ["ra","dec"]:
        if word+"_image" in param:
            return kw_models["ps_params"],"_image"
    if "lens_light" in param: 
        # this should catch if there are "center_x/y_lens_light"
        # so that i consider the correct prior
        return kw_models["lens_light_params"],"_lens_light"
    if "source_light" in param:
        try:
            return kw_models["source_params"],"_source_light"
        except KeyError:
            raise KeyError("The given setting file do not have source params, but the resulting mcmc have them")
    
    if "center_x_lens" in param or "center_y_lens" in param:
        return kw_models["lens_params"],"_lens"
    # if it doesn't manage to find anything, creates an error
    return None
    
def index_image(param,ip,nparam,n_images=4):
    # get the index of the image
    n_starting = nparam - 2*n_images #n_images*ra,n_images*dec
    if "ra_" in param:
        return int(ip-n_starting)
    elif "dec_" in param:
        return int(ip-n_starting-n_images)
    else:
        raise RuntimeError("These params should be image")

In [ ]:
def Df_prior(setting_name,npoints=10000,Df_boundaries=None,save_mcmc=False,backup_path = "./backup_results/",output_name="mcmc_Df_prior"):
    mcmc_path = backup_path+"/"+setting_name.replace("settings","mcmc").replace(".py","")
    mcmc_prior_path = mcmc_path+"/"+output_name+".json"

    setting_module = get_setting_module(setting_name) 
    setting        = setting_module.setting()
    
    while True:
        mcmc_prior,param_mcmc = get_prior(setting_module,npoints)
        try:
            mcmc_Df_prior  = get_mcmc_Df_prior(mcmc_prior,param_mcmc,setting,Df_boundaries)
            break
        except UserWarning:
            print("Increasing n_points of 70000")
            npoints       += 70000
    print("Df prior Done")
    if save_mcmc:
        with open(mcmc_prior_path,"w") as f:
            json.dump(np.array(mcmc_Df_prior).tolist(),f)
    return mcmc_Df_prior

In [ ]:
def Df_prior_ABC(setting_name,npoints=10000,Df_boundaries=None,save_mcmc=False,backup_path = "./backup_results/",output_name="mcmc_Df_prior_ABC"):
    # Consider ABC but not D
    mcmc_path = backup_path+"/"+setting_name.replace("settings","mcmc").replace(".py","")
    mcmc_prior_path = mcmc_path+"/"+output_name+".json"
    mcmc_Df_prior = Df_prior(setting_name,npoints,Df_boundaries=Df_boundaries,save_mcmc=False,backup_path=backup_path,output_name=output_name)
    
    print("WARNING: Only considering ABC")    
    mcmc_c    = np.array(copy.deepcopy(mcmc_Df_prior))
    # BC = C - B = (C-A)-(B-A) = AC - AB
    mcmc_BC   = mcmc_c[1]- mcmc_c[0]
    mcmc_c[2] = mcmc_BC
    mcmc_Df_prior_ABC = mcmc_c
    
    if save_mcmc:
        with open(mcmc_prior_path,"w") as f:
            json.dump(mcmc_Df_prior_ABC.tolist(),f)
    return mcmc_Df_prior_ABC

In [ ]:
def mag_prior(setting_name,npoints=10000,mag_boundaries=None,save_mcmc=False,backup_path = "./backup_results/",output_name="mcmc_mag_rt_prior"):
    mcmc_path = backup_path+"/"+setting_name.replace("settings","mcmc").replace(".py","")
    mcmc_prior_path = mcmc_path+"/"+output_name+".json"

    setting_module = get_setting_module(setting_name) 
    setting        = setting_module.setting()
    
    while True:
        mcmc_prior,param_mcmc = get_prior(setting_module,npoints)
        try:
            mcmc_mag_prior  = get_mcmc_mag_prior(mcmc_prior,param_mcmc,setting,mag_boundaries)
        except UserWarning:
            print("Increasing n_points of 70000")
            npoints       += 70000
    print("Mag prior Done")
    if save_mcmc:
        with open(mcmc_prior_path,"w") as f:
            json.dump(np.array(mcmc_mag_prior).tolist(),f)
    return mcmc_mag_prior

def mag_prior_ABC(setting_name,npoints=10000,mag_boundaries=None,save_mcmc=False,backup_path = "./backup_results/",output_name="mcmc_mag_rt_prior_ABC"):
    # Consider ABC but not D
    mcmc_path = backup_path+"/"+setting_name.replace("settings","mcmc").replace(".py","")
    mcmc_prior_path = mcmc_path+"/"+output_name+".json"
    mcmc_mag_prior = mag_prior(setting_name,npoints,mag_boundaries=mag_boundaries,save_mcmc=False,backup_path=backup_path,output_name=output_name)
    
    print("WARNING: Only considering ABC")    
    mcmc_c    = np.array(copy.deepcopy(mcmc_mag_prior))
    # BC = C / B = (C/A)/(B/A) = AC/AB
    mcmc_BC   = mcmc_c[1]/mcmc_c[0]
    mcmc_c[2] = mcmc_BC
    mcmc_mag_prior_ABC = mcmc_c
    
    if save_mcmc:
        with open(mcmc_prior_path,"w") as f:
            json.dump(mcmc_mag_prior_ABC.tolist(),f)
    return mcmc_mag_prior_ABC

In [ ]:
if __name__=="__main__":
    parser = argparse.ArgumentParser(description="Produces the prior of the difference of Fermat potential differences given the setting file")
    parser.add_argument("-PP", "--plot_prior", action="store_true", dest="plot_prior", default=False,
                        help="Plot the prior distribution for the Fermat potential difference at the prior image positions")
    parser.add_argument("-ABCD", action="store_true", dest="ABCD", default=False,
                        help="Consider images A,B, C and also D (usually ignored due to low S/N)")
    parser.add_argument("-np", "--npoints", type=int,dest="npoints", default=1000,
                        help="Number of points for the MCMC prior")
    parser.add_argument("-DfB", "--Df_boundaries",dest="Df_boundaries", default=None,
                        help="Boundaries of the DFermat potential space")
    parser.add_argument('SETTING_FILES',nargs="+",default=[],help="setting file(s) to consider")
    args = parser.parse_args()
    npoints  = args.npoints
    settings = args.SETTING_FILES
    plot_prior = args.plot_prior
    ABCD = args.ABCD
    Df_boundaries = args.Df_boundaries
    ####################
    present_program(sys.argv[0])
    ####################
    backup_path   = "./backup_results/"
    
    labels = ["$\Delta\phi_{AB}$","$\Delta\phi_{AC}$","$\Delta\phi_{AD}$"]
    prior_name = "mcmc_Df_prior"
    if not ABCD:
        labels[-1] = "$\Delta\phi_{BC}$"
        prior_name = prior_name+"_ABC"
    for sets in settings:
        print(sets,":")
        if not ABCD:
            Df_prior_i = Df_prior_ABC(sets.replace(".py",""),npoints,save_mcmc=True,Df_boundaries=Df_boundaries,backup_path=backup_path,output_name=prior_name)
        else:    
            Df_prior_i = Df_prior(sets.replace(".py",""),npoints,save_mcmc=True,Df_boundaries=Df_boundaries,backup_path=backup_path,output_name=prior_name)
        if plot_prior:
            savefig_path = get_savefigpath(sets,backup_path)
            corner.corner(Df_prior_i,labels=labels,show_titles=True)
            plt.savefig(savefig_path+"/"+prior_name+".png")
            plt.close()
    success(sys.argv[0])